In [1]:
!pip install --upgrade "transformers>=4.38" accelerate bitsandbytes sentencepiece --quiet

ERROR: Operation cancelled by user


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch, pandas as pd, pathlib, datetime, textwrap
from tqdm.auto import tqdm

MODEL_ID = "IlyaGusev/saiga_mistral_7b_lora"

tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            device_map="auto",
            torch_dtype=torch.float16,
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True
        ).eval()

streamer = TextStreamer(tok)


In [ ]:
import pandas as pd, pathlib, chardet, csv, io

path = pathlib.Path("/content/all_reviews.csv")

encoding = "windows-1251"

sample = path.read_bytes()[:32_000].decode(encoding, errors="replace")
sniff = csv.Sniffer().sniff(sample, delimiters=",;\t|")
dialect_delim = sniff.delimiter
print("Detected delimiter:", repr(dialect_delim))

df = pd.read_csv(
        path,
        encoding=encoding,
        sep=dialect_delim,
        engine="python",
        quoting=csv.QUOTE_NONE,
        on_bad_lines="skip",
    )

print("✅ загружено строк:", len(df))
print("Колонки:", list(df.columns)[:10])


In [ ]:
neg = df.loc[df["sentiment"].isin(["negative", "neutral"]), "text"] \
            .astype(str) \
            .tolist()

print("Негативных отзывов:", len(neg))


In [ ]:
COMPANY_CTX = textwrap.dedent("""
Компания: «Передовые Платёжные Решения» (ППР)
Тип бизнеса: финтех-оператор B2B
Клиенты: корпоративные автопарки и командировочные службы (≈80k компаний)
Услуги: оплата топлива, штрафы, парковки, платные дороги, отчётность по расходам
Цель: снижать негатив клиентов, улучшать поддержку, ускорять возвраты, держать тарифы конкурентными
""").strip()

FEWSHOT = textwrap.dedent("""
### Отзывы:
- Слишком долго оформляют возврат.
- Операторы не помогают решить вопрос.

### Рекомендации:
1. Сократить срок возврата средств до 5 дней.
2. Ввести KPI операторов: не менее 80 % решённых обращений.
""").strip()

FORBID = ("Не предлагай уходить к конкурентам. "
          "Не предлагай закрыть счёт или заменить компанию.")

def prompt(batch):
    system = (
        "Ты — эксперт по клиентскому опыту. Я — менеджер компании ППР."
        "Дай мне до 10 конкретных рекомендаций, как снизить негатив. "
        "Не предлагай уходить к конкурентам или искать альтернативу."
        f"{FORBID} Формат ответа: нумерованный список 4–6 пунктов."
        "\n\nКонтекст компании и клиентов:\n" + COMPANY_CTX
    )
    user = "\n".join(f"- {t.strip()}" for t in batch)
    return f"<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n{FEWSHOT}\n\n### Отзывы:\n{user}\n\n### Рекомендации: [/INST]"


In [ ]:
from tqdm.auto import tqdm
import textwrap, torch

BATCH_SIZE = 8

def batches(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

# def strip_banned(text: str) -> str:
#     keep = [ln for ln in text.splitlines()
#             if not any(bad in ln.lower() for bad in BAD_PHRASES)]
#     return "\n".join(keep).strip() or text.strip()

def generate_recommendations(reviews, batch=BATCH_SIZE):
    output = []
    for pack in tqdm(list(batches(reviews, batch)), desc="☁️ генерация", unit="batch"):
        inp = tok(prompt(pack), return_tensors="pt").to(model.device)
        with torch.no_grad():
            gen = model.generate(
                **inp,
                do_sample=True,
                temperature=0.8,
                top_p=0.9,
                repetition_penalty=1.15,
                no_repeat_ngram_size=4,
                max_new_tokens=256,
            )
        text = tok.decode(gen[0, inp.input_ids.shape[1]:],
                          skip_special_tokens=True).strip()
        # output.append(strip_banned(text))
        output.append(text)
    return output

reviews_src = neg if "neg_clean" in globals() else neg
advices = generate_recommendations(reviews_src)

print("\nПример рекомендаций:\n")
print(advices[0])
